# Trading Performance Reporter - Example Usage

This notebook demonstrates how to use the Trading Performance Reporter package to generate comprehensive reports for your Hyperliquid trading account.

## Features

- **AUM Tracking**: Monitor your Assets Under Management over time
- **Performance Analysis**: Track P&L in both USD and percentage
- **Trade Analysis**: Detailed breakdown of all trades with costs and fees
- **Funding Costs**: Track funding payments for perpetual positions
- **Professional Reports**: Generate HTML reports with visualizations
- **Email Delivery**: Automatically send reports via email

## Setup

First, import the necessary modules:

In [1]:
import logging
from datetime import datetime, timedelta
from pathlib import Path

from config import load_config
from hyperliquid_reporter.monitoring import HyperliquidMonitor
from hyperliquid_reporter.reporter import HyperliquidReporter

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 1. Load Configuration

Load your Hyperliquid account configuration:

In [2]:
config = load_config(
    config_path="config_hyperliquid.json",
    testnet=False
)

account_address = config['account_address']
info = config['_hyperliquid_info']

print(f"Account: {account_address[:10]}...{account_address[-8:]}")
print(f"Network: {'Testnet' if config['testnet'] else 'Mainnet'}")

INFO:config:Initializing Hyperliquid via token_data.setup()
INFO:config:Config file: config_hyperliquid.json
INFO:config:Network: mainnet
INFO:config:Base URL: https://api.hyperliquid.xyz
INFO:config:Successfully initialized Hyperliquid for account: 0x143E18B5...


Account: 0x143E18B5...54F66cAa
Network: Mainnet


## 2. Initialize Monitor

Create a monitor to fetch account data:

In [3]:
monitor = HyperliquidMonitor(info=info, address=account_address)
print("Monitor initialized successfully!")

INFO:hyperliquid_reporter.monitoring:Initialized HyperliquidMonitor for account 0x143E18B5...


Monitor initialized successfully!


## 3. Get Current Metrics

Fetch current account metrics:

In [4]:
metrics = monitor.get_current_metrics()
print(f"Account Value: ${metrics.account_value:,.2f}")
print(f"Unrealized P&L: ${metrics.unrealized_pnl:,.2f}")

Account Value: $51.27
Unrealized P&L: $2.25


## 4. Get Account Summary

Get comprehensive account summary including deposits and withdrawals:

In [5]:
account_summary = monitor.get_account_summary(lookback_days=3650)

print(f"Total Deposits: ${account_summary['total_deposits']:,.2f}")
print(f"Total Withdrawals: ${account_summary['total_withdrawals']:,.2f}")
print(f"Net Deposits: ${account_summary['net_deposits']:,.2f}")
print(f"Current Value: ${account_summary['current_value']:,.2f}")
print(f"Total P&L: ${account_summary['total_pnl']:,.2f}")
print(f"P&L %: {account_summary['pnl_percentage']:.2f}%")
print(f"Spot Value: ${account_summary['spot_value']:,.2f}")
print(f"Perp Value: ${account_summary['perp_value']:,.2f}")

Error retrieving user ledger updates: 'usdc'
PnL: {'@151': -6.81, 'ETH': 6.38}


ERROR:hyperliquid_reporter.monitoring:token_data account summary failed; falling back to Info-based summary
Traceback (most recent call last):
  File "/Users/christiansilva/Documents/Tata/Projects/Code/reporter/hyperliquid_reporter/monitoring.py", line 677, in get_account_summary
    raise TypeError("token_data.get_account_summary returned None")
TypeError: token_data.get_account_summary returned None


Error retrieving account summary: name 'spot_tickers_here' is not defined


INFO:hyperliquid_reporter.monitoring:Spot: total=$48.15, USDC=$17.69, non-USDC current=$30.46, entry=$37.31, unrealized=$-6.85
INFO:hyperliquid_reporter.monitoring:Total account value: $99.40, Perp: $51.25, Spot: $48.15, Unrealized P&L: $-4.61
INFO:hyperliquid_reporter.monitoring:Fetched ledger data: deposits=$99.41, withdrawals=$0.00


Total Deposits: $99.41
Total Withdrawals: $0.00
Net Deposits: $99.41
Current Value: $99.40
Total P&L: $-0.01
P&L %: -0.01%
Spot Value: $48.15
Perp Value: $51.25


## 5. Initialize Reporter

Create a reporter to generate reports:

In [6]:
reporter = HyperliquidReporter(monitor=monitor, account_address=account_address)
print("Reporter initialized successfully!")

INFO:hyperliquid_reporter.reporter:Initialized HyperliquidReporter for account 0x143E18B5...


Reporter initialized successfully!


## 6. Generate AUM Data

Get Assets Under Management over time:

In [7]:
aum_data = reporter.generate_aum_data(period="allTime")
print(f"AUM Data Points: {len(aum_data)}")
if not aum_data.empty:
    print(f"\nFirst 5 entries:")
    print(aum_data.head())
    print(f"\nLast 5 entries:")
    print(aum_data.tail())

AUM Data Points: 35

First 5 entries:
                           aum_usd
timestamp                         
2025-09-17 22:42:00.060   0.000000
2025-09-17 23:42:00.060  29.979257
2025-09-24 23:42:00.013  29.912292
2025-10-01 23:02:00.057  29.957827
2025-10-08 21:42:00.094  30.009890

Last 5 entries:
                           aum_usd
timestamp                         
2026-01-23 14:42:00.025  91.269696
2026-01-24 23:42:00.019  99.397290
2026-01-26 07:42:00.052  99.397535
2026-01-27 16:42:00.079  99.416255
2026-01-28 14:40:21.115  99.413154


## 7. Generate Performance Data

Get performance metrics (P&L in $ and %):

In [8]:
performance_data = reporter.generate_performance_data(period="allTime")
print(f"Performance Data Points: {len(performance_data)}")
if not performance_data.empty:
    print(f"\nLast 5 entries:")
    print(performance_data.tail())

INFO:hyperliquid_reporter.reporter:Reconstructed net deposits timeline with 7 deposit/withdrawal events


Performance Data Points: 35

Last 5 entries:
                           aum_usd  net_deposits   pnl_usd   pnl_pct
timestamp                                                           
2026-01-23 14:42:00.025  91.269696         91.26  0.000644  0.000706
2026-01-24 23:42:00.019  99.397290         99.41 -0.022406 -0.024549
2026-01-26 07:42:00.052  99.397535         99.41  0.000245  0.000246
2026-01-27 16:42:00.079  99.416255         99.41  0.018720  0.018833
2026-01-28 14:40:21.791  99.413154         99.41 -0.003101 -0.003119


In [9]:
performance_data

,aum_usd,net_deposits,pnl_usd,pnl_pct
timestamp,,,,
2025-09-17 22:42:00.060,30.000000,30.00,0.000000,0.000000
2025-09-17 23:42:00.060,29.979257,30.00,-0.020743,-0.069143
2025-09-24 23:42:00.013,29.912292,30.00,-0.066965,-0.223371
2025-10-01 23:02:00.057,29.957827,30.00,0.045535,0.152228
2025-10-08 21:42:00.094,30.009890,30.00,0.052063,0.173788
2025-10-15 21:42:00.079,29.916767,30.00,-0.093123,-0.310308
2025-10-22 22:42:00.065,61.770629,61.90,-0.046138,-0.154221
2025-10-29 23:42:00.055,61.826730,61.90,0.056101,0.090821
2025-11-05 21:42:00.031,91.002780,91.26,-0.183950,-0.297525


## 8. Generate Trade Analysis

Analyze all trades with costs and fees:

In [13]:
end_time = datetime.now()
start_time = end_time - timedelta(days=90)

trade_analysis = reporter.generate_trade_analysis(
    start_time=start_time,
    end_time=end_time
)

print(f"Total Trades (last 30 days): {len(trade_analysis)}")
if not trade_analysis.empty:
    print(f"\nTrade Summary:")
    print(f"Total Volume: ${trade_analysis['notional'].sum():,.2f}")
    print(f"Total Fees: ${trade_analysis['fee'].sum():,.2f}")
    print(f"Total Net P&L: ${trade_analysis['net_pnl'].sum():,.2f}")
    print(f"\nLast 10 trades:")
    print(trade_analysis.tail(10))

Total Trades (last 30 days): 4

Trade Summary:
Total Volume: $49.19
Total Fees: $0.02
Total Net P&L: $4.13

Last 10 trades:
                         coin  side   price    size  notional       fee  \
datetime                                                                  
2025-11-04 21:41:13.734   ETH   buy  3108.1  0.0035  10.87835  0.004895   
2025-11-04 21:42:37.897   ETH  sell  3088.4  0.0036  11.11824  0.005003   
2026-01-24 14:39:08.729   ETH  sell  2957.2  0.0046  13.60312  0.006121   
2026-01-24 14:43:04.837  @151   buy  2955.1  0.0046  13.59346  0.000003   

                         closed_pnl   net_pnl  
datetime                                       
2025-11-04 21:41:13.734    4.150265  4.145370  
2025-11-04 21:42:37.897    0.000000 -0.005003  
2026-01-24 14:39:08.729    0.000000 -0.006121  
2026-01-24 14:43:04.837    0.000000 -0.000003  


## 9. Generate Funding Analysis

Analyze funding costs for perpetual positions:

In [14]:
funding_analysis = reporter.generate_funding_analysis(
    start_time=start_time,
    end_time=end_time,
    lookback_days=90
)

print(f"Funding Payments (last 30 days): {len(funding_analysis)}")
if not funding_analysis.empty:
    print(f"\nFunding Summary:")
    print(f"Total Funding: ${funding_analysis['funding_payment'].sum():,.4f}")
    print(f"\nFunding by Coin:")
    funding_by_coin = funding_analysis.groupby('coin')['funding_payment'].sum().sort_values()
    print(funding_by_coin)
    print(f"\nLast 10 funding payments:")
    print(funding_analysis.tail(10))

Funding Payments (last 30 days): 283

Funding Summary:
Total Funding: $0.3488

Funding by Coin:
coin
ETH    0.348763
Name: funding_payment, dtype: float64

Last 10 funding payments:
                        coin  funding_payment  position_size  funding_rate
datetime                                                                  
2026-01-28 00:00:00.090  ETH         0.000378          -0.01      0.000013
2026-01-28 01:00:00.024  ETH         0.000377          -0.01      0.000013
2026-01-28 02:00:00.053  ETH         0.000377          -0.01      0.000013
2026-01-28 03:00:00.047  ETH         0.000376          -0.01      0.000013
2026-01-28 04:00:00.114  ETH         0.000375          -0.01      0.000013
2026-01-28 05:00:00.046  ETH         0.000374          -0.01      0.000013
2026-01-28 06:00:00.127  ETH         0.000375          -0.01      0.000013
2026-01-28 07:00:00.070  ETH         0.000376          -0.01      0.000013
2026-01-28 08:00:00.044  ETH         0.000375          -0.01      0.

## 10. Generate Complete Report

Generate all report data at once:

In [ ]:
report_data = reporter.generate_report_data(
    period="allTime",
    start_time=start_time,
    end_time=end_time,
    lookback_days=30
)

print("Report Data Generated!")
print(f"\nSummary Statistics:")
stats = report_data['summary_stats']
for key, value in stats.items():
    if isinstance(value, (int, float)):
        if 'pct' in key:
            print(f"{key}: {value:.2f}%")
        elif isinstance(value, int):
            print(f"{key}: {value:,}")
        else:
            print(f"{key}: ${value:,.2f}")

## 11. Create Visualizations

Generate charts and graphs:

In [ ]:
visualizations = reporter.create_visualizations(
    report_data=report_data,
    output_dir="reports"
)

print(f"Created {len(visualizations)} visualizations:")
for name in visualizations.keys():
    print(f"  - {name}")

## 12. Generate HTML Report

Create a professional HTML report:

In [ ]:
html_content = reporter.generate_html_report(
    report_data=report_data,
    visualizations=visualizations
)

output_path = Path("reports")
output_path.mkdir(parents=True, exist_ok=True)

report_filename = f"trading_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
report_path = output_path / report_filename

with open(report_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"HTML report saved to: {report_path}")
print(f"Report size: {len(html_content):,} bytes")

## 13. Send Report via Email (Optional)

Send the report via email using the mail_it function:

In [ ]:
from mail_it import mail_it

result = mail_it(
    to="silvaac@yahoo.com",
    subj=f"Trading Performance Report - {datetime.now().strftime('%Y-%m-%d')}",
    body=html_content
)

if result == 0:
    print("✓ Report sent successfully!")
else:
    print("✗ Failed to send report")

## Advanced Usage Examples

### Get Portfolio History for Different Periods

In [ ]:
periods = ["day", "week", "month", "allTime"]

for period in periods:
    df = monitor.get_portfolio_dataframe(period=period)
    if not df.empty:
        print(f"\n{period.upper()}:")
        print(f"  Data points: {len(df)}")
        if 'account_value' in df.columns:
            print(f"  Latest AUM: ${df['account_value'].iloc[-1]:,.2f}")
        if 'pnl' in df.columns:
            print(f"  Latest P&L: ${df['pnl'].iloc[-1]:,.2f}")

### Analyze Trade Performance by Coin

In [ ]:
if not trade_analysis.empty:
    print("Trade Performance by Coin:\n")
    
    by_coin = trade_analysis.groupby('coin').agg({
        'notional': 'sum',
        'fee': 'sum',
        'net_pnl': 'sum',
        'coin': 'count'
    }).rename(columns={'coin': 'trade_count'})
    
    by_coin = by_coin.sort_values('notional', ascending=False)
    print(by_coin)

### Calculate Win Rate by Coin

In [ ]:
if not trade_analysis.empty:
    print("Win Rate by Coin:\n")
    
    def calc_win_rate(group):
        total = len(group)
        wins = len(group[group['net_pnl'] > 0])
        return (wins / total * 100) if total > 0 else 0
    
    win_rates = trade_analysis.groupby('coin').apply(calc_win_rate)
    win_rates = win_rates.sort_values(ascending=False)
    
    for coin, rate in win_rates.items():
        print(f"{coin}: {rate:.1f}%")

## Conclusion

This notebook demonstrated all the key features of the Trading Performance Reporter:

1. ✅ Loading configuration and initializing the monitor
2. ✅ Fetching current metrics and account summary
3. ✅ Generating AUM data over time
4. ✅ Analyzing performance (P&L in $ and %)
5. ✅ Analyzing trades with costs and fees
6. ✅ Tracking funding costs for perpetuals
7. ✅ Creating visualizations
8. ✅ Generating professional HTML reports
9. ✅ Sending reports via email

For automated reporting, use the `run_report.py` script:

```bash
python run_report.py
```

You can customize the parameters in `run_report.py` to suit your needs.